## **Multi-Agent DeelSeeker - Modal Research Notebook**

This notebook documents key steps in the research process for building the Multi-Agent DeelSeeker project.
It includes code for integrating Modal services, deploying AI agents, and interacting with Llama models.

### **Introduction to Modal**  

[Modal](https://modal.com/docs) is a serverless computing platform that allows developers to build, deploy, and run machine learning and AI applications in the cloud without managing infrastructure. It provides a seamless way to run Python code in the cloud, making it ideal for deploying ML models, APIs, and background jobs efficiently.  


### **Key Features of Modal**  

1. **Serverless Execution**  
   - No need to manage servers or cloud instances; Modal automatically handles resource allocation.  
   
2. **Function and Class-Based Deployment**  
   - Supports both function-level execution (`modal.Function`) and class-based services (`modal.Cls`).  

3. **Efficient Scaling**  
   - Automatically scales resources based on workload, making it cost-effective for AI applications.  

4. **Seamless Integration with Python**  
   - Allows you to deploy Python scripts and modules with minimal changes using `modal.App`.  

5. **Secure Secret Management**  
   - Securely store and use API keys (e.g., Hugging Face, OpenAI) using Modal’s built-in secret management.  

6. **Region-Specific Execution**  
   - Deploy workloads to different geographical regions for optimized performance.  

---

### **Basic Workflow in Modal**  

1. **Install Modal**  
   ```bash
   pip install modal
   ```

2. **Create a Modal App**  
   ```python
   import modal
   
   app = modal.App("my_modal_app")
   ```

3. **Define and Deploy a Function**  
   ```python
   @app.function()
   def greet(name: str):
       return f"Hello, {name}!"
   ```

4. **Run the Function Locally or Remotely**  
   ```python
   with app.run():
       response = greet.remote("Alice")
       print(response)
   ```

5. **Deploy the App**  
   ```bash
   modal deploy my_modal_app.py
   ```

---

### **Function vs. Module Deployment**  

- **Script-Based Deployment:**  
  ```bash
  modal deploy my_service.py
  ```  
  This deploys `my_service.py` as a standalone Modal service, allowing function lookup using `modal.Function.lookup()`.

- **Module-Based Deployment (`-m` Flag):**  
  ```bash
  modal deploy -m my_package.my_service
  ```  
  This deploys `my_service` as a module within a package, enabling class-based lookup using `modal.Cls.lookup()`.

### **Official Documentation**  
For detailed guides and API references, visit: [Modal Documentation](https://modal.com/docs).  

Let me know if you need a more tailored explanation!

In [ ]:
import modal

In [ ]:
# Uncomment and run the line below if you haven't set up Modal on your machine
# !modal setup

### **Importing and Running the Modal App**


In [ ]:
# Local Run

from hello import app, hello, hello_europe

with app.run():
    test_output = hello.local()
test_output

In [ ]:
# Runing remotely

with app.run():
    test_output = hello.remote()
test_output

### **Configuring Hugging Face Token to work with Modal**

To use Hugging Face models with Modal, set up a secret token:
1. Sign in to modal.com
2. Navigate to the Secrets section in the dashboard
3. Create a new secret named `hf-secret` and store your Hugging Face API key there.

In [ ]:
## Working with Llama Model

from llama import app, generate

with modal.enable_output():
    with app.run():
        result = generate.remote("Life is a mystery, everyone must stand alone, I hear")
result

In [ ]:
## Pricing API via Modal

from DealSeeker_pricer_ephemeral import app, price

with modal.enable_output():
    with app.run():
        result = price.remote("Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio")
result

### **Deploying Modal Services**



In [ ]:
!modal deploy DealSeeker_pricer_service.py


In [ ]:
pricer = modal.Function.lookup("pricer-service", "price")
response = pricer.remote("Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio")
response


In [ ]:
!modal deploy -m DealSeeker_pro_pricer_service.py


In [ ]:
Pricer = modal.Cls.lookup("pricer-service", "Pricer")
pricer_instance = Pricer()
response = pricer_instance.price.remote("Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio")
response

### **Observations:**

- **Without `-m` (!modal deploy DealSeeker_pricer_service.py)**: Deploys `DealSeeker_pricer_service.py` as a standalone Modal service, allowing function lookup via `modal.Function.lookup()`. The `price` function is called remotely.  

- **With `-m` (!modal deploy -m DealSeeker_pro_pricer_service.py)**: Deploys `DealSeeker_pro_pricer_service.py` as a Python module, allowing class-based lookup via `modal.Cls.lookup()`. An instance of `Pricer` is created to call the `price` method remotely.

---

### **Optional: Keeping Modal Services Warm**

- Prevent services from idling to improve response time.
- Use `keep_warm.py` to ping the service every 30 seconds.

    **Run in a separate terminal:**

    ```python
    python keep_warm.py
    ```

---

### **Implementing the Specialist Agent**

In [ ]:
!pip install --upgrade modal

In [ ]:
from agents.specialist_agent import SpecialistAgent

agent = SpecialistAgent()
result = agent.price("iPad Pro 2nd generation")
result